## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


In [2]:
#!pip install sqlalchemy==1.3.9

### Connect to the database

In [3]:
#!pip install ipython-sql
#!pip install ipython-sql prettytable

In [4]:
%load_ext sql

In [5]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [6]:
#!pip install -q pandas

In [7]:
%sql sqlite:///my_data1.db

In [8]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
#from sqlalchemy import create_engine
#engine = create_engine("sqlite:///my_data1.db")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

101

In [9]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [10]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

##### 1. Display the names of the unique launch sites  in the space mission


In [11]:
%%sql
SELECT DISTINCT "Launch_Site"
FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [12]:
%%sql
SELECT DISTINCT "Launch_Site", "Mission_Outcome"
FROM SPACEXTABLE
WHERE "Mission_Outcome" LIKE "Success";

 * sqlite:///my_data1.db
Done.


Launch_Site,Mission_Outcome
CCAFS LC-40,Success
VAFB SLC-4E,Success
KSC LC-39A,Success
CCAFS SLC-40,Success


### 2. Display 5 records where launch sites begin with the string 'CCA'

In [13]:
%%sql
SELECT *
FROM SPACEXTABLE
WHERE "Launch_Site" LIKE "CCA%"
LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### 3. Display the total payload mass carried by boosters launched by NASA (CRS).

In [14]:
%%sql
SELECT SUM("PAYLOAD_MASS__KG_") AS "Total_Payload_Mass_NASA_CRS"
FROM SPACEXTABLE
WHERE "Customer" LIKE "NASA (CRS)";

 * sqlite:///my_data1.db
Done.


Total_Payload_Mass_NASA_CRS
45596


### 4. Display average payload mass carried by booster version F9 v1.1

In [15]:
%%sql
SELECT AVG("PAYLOAD_MASS__KG_") AS "Avg_Payload_Mass_F9v1.1"
FROM SPACEXTABLE
WHERE "Booster_Version" LIKE "F9 v1.1%";

 * sqlite:///my_data1.db
Done.


Avg_Payload_Mass_F9v1.1
2534.6666666666665


### 5. List the date when the first succesful landing outcome in ground pad was acheived.

In [16]:
%%sql
SELECT DISTINCT Landing_Outcome
FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Landing_Outcome
Failure (parachute)
No attempt
Uncontrolled (ocean)
Controlled (ocean)
Failure (drone ship)
Precluded (drone ship)
Success (ground pad)
Success (drone ship)
Success
Failure


In [17]:
%%sql
SELECT "DATE"
FROM SPACEXTABLE
WHERE "Landing_Outcome" LIKE "Success (ground pad)"
ORDER BY "DATE" ASC
LIMIT 1;

 * sqlite:///my_data1.db
Done.


Date
2015-12-22


### 6. List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [18]:
%%sql
SELECT "Booster_Version"
FROM SPACEXTABLE
WHERE "Landing_Outcome" LIKE "Success (drone ship)" AND 
    "PAYLOAD_MASS__KG_" > 4000 AND "PAYLOAD_MASS__KG_" < 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


### 7. List the total number of successful and failure mission outcomes

In [19]:
%%sql
SELECT 
    CASE
        WHEN "Mission_Outcome" LIKE "Failure%" THEN "Failure"
        ELSE "Success" 
    END AS "MissionOutcomes",
    COUNT(*) AS "Number of Outcomes"
FROM SPACEXTABLE
GROUP BY "MissionOutcomes";

 * sqlite:///my_data1.db
Done.


MissionOutcomes,Number of Outcomes
Failure,1
Success,100


In [20]:
%%sql
SELECT "Mission_Outcome", COUNT(*)
FROM SPACEXTABLE
GROUP BY "Mission_Outcome";

 * sqlite:///my_data1.db
Done.


Mission_Outcome,COUNT(*)
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


### 8. List the names of the booster_versions which have carried the maximum payload mass.

In [21]:
%%sql
SELECT *
FROM SPACEXTABLE
WHERE Launch_Site LIKE "CCA%"
LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [22]:
%%sql
SELECT "Booster_Version", "PAYLOAD_MASS__KG_"
FROM SPACEXTABLE
WHERE "PAYLOAD_MASS__KG_" == (SELECT MAX("PAYLOAD_MASS__KG_") FROM SPACEXTABLE);

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


In [23]:
%%sql
SELECT MAX("PAYLOAD_MASS__KG_")
FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


"MAX(""PAYLOAD_MASS__KG_"")"
15600


### 9. List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [24]:
%%sql
SELECT 
    CASE strftime('%m', "DATE")
        WHEN '01' THEN 'January'
        WHEN '02' THEN 'February'
        WHEN '03' THEN 'March'
        WHEN '04' THEN 'April'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'June'
        WHEN '07' THEN 'July'
        WHEN '08' THEN 'August'
        WHEN '09' THEN 'September'
        WHEN '10' THEN 'October'
        WHEN '11' THEN 'November'
        WHEN '12' THEN 'December'
    END AS "Month",
    "Landing_Outcome",
    "Booster_Version",
    "Launch_Site"
FROM SPACEXTABLE
WHERE strftime('%Y', "DATE") = '2015' AND
    "Landing_Outcome" LIKE "Failure (drone ship)";

 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [25]:
%%sql
SELECT 
    SUBSTR("Date", 6, 2) AS "Month",
    "Landing_Outcome",
    "Booster_Version",
    "Launch_Site"
FROM SPACEXTABLE
WHERE 
    SUBSTR("Date", 1, 4) = '2015'
    AND "Landing_Outcome" LIKE 'Failure (drone ship)';

 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### 10. Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [26]:
%%sql
SELECT 
    RANK() OVER (ORDER BY COUNT(*) DESC) AS Rank,
    "Landing_Outcome",
    COUNT("Landing_Outcome") AS "OutcomeCount"
FROM SPACEXTABLE
WHERE DATE > '2010-06-04' AND DATE < '2017-03-20' AND
    "Landing_Outcome" IN("Failure (drone ship)", "Success (ground pad)")
GROUP BY "Landing_Outcome"
ORDER BY Rank DESC;

 * sqlite:///my_data1.db
Done.


Rank,Landing_Outcome,OutcomeCount
2,Success (ground pad),3
1,Failure (drone ship),5


In [27]:
%%sql
SELECT MIN(PAYLOAD_MASS__KG_)
FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


MIN(PAYLOAD_MASS__KG_)
0
